In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from delta.tables import DeltaTable

# **Create Delta Table**

### **Using SQL API**

In [0]:
%sql
CREATE TABLE deltatable.default.deltaapi
(
  id INT NOT NULL,
  salary INT
)

### **Using Delta API**

In [0]:
from delta.tables import DeltaTable

In [0]:
DeltaTable.createIfNotExists(spark) \
  .tableName("deltatable.default.deltaapi") \
  .addColumn("id", "INT") \
  .addColumn("salary", "INT") \
  .execute()

### **Computed Columns**

In [0]:
DeltaTable.create(spark)\
  .tableName("deltatable.default.deltaapi_1")\
  .addColumn("salaryAfterTax", dataType=LongType(), generatedAlwaysAs = "CAST((salary * 0.7) AS BIGINT)")\
  .addColumn("salary", dataType = IntegerType())\
  .addColumn("name", dataType = StringType())\
  .execute()


In [0]:
%sql
INSERT INTO deltatable.default.deltaapi_1(salary,name)
VALUES 
(100,'aa'),
(200,'bb')

In [0]:
%sql
SELECT * FROM deltatable.default.deltaapi_1

# **KEY FEATURES DELTA LOG**

In [0]:
data = [(1,100,'aa'),(2,200,'bb'),(3,300,'cc'),(4,400,'dd')]

df = spark.createDataFrame(data, ['id','salary','name'])

display(df)

In [0]:
df.write.format("delta")\
        .mode("append")\
        .save("/Volumes/deltatable/default/deltavol/logsink")

In [0]:
df = spark.read.format("json")\
            .load("/Volumes/deltatable/default/deltavol/logsink/_delta_log/00000000000000000000.json")
display(df)

# **SCHEMA ENFORCEMENT**


In [0]:
data = [(1,100,'aa',1),(2,200,'bb',1),(3,300,'cc',1),(4,400,'dd',1)]

df = spark.createDataFrame(data, ['id','salary','name','sus_col'])

display(df)

In [0]:
df.write.format("delta")\
        .mode("append")\
        .save("/Volumes/deltalakeansh/default/deltavol/Logsink/")

In [0]:
#schema Evolution
df.write.format("delta")\
        .mode("append")\
        .option("mergeSchema",True)\
        .save("/Volumes/deltalakeansh/default/deltavol/Logsink/")

In [0]:
SELECT * FROM deltatable.default.deltaapi_1

# READ DELTA DATA

In [0]:
%sql
SELECT * FROM deltatable.default.deltaapi_1

Lake

In [0]:
%sql
SELECT * FROM delta.`/Volumes/deltalakeansh/default/deltavol/logsink/`

In [0]:
data = [(1,100,'aa',1),(2,200,'bb',1),(3,300,'cc',1),(4,400,'dd',1)]

df = spark.createDataFrame(data, ['cust_id','income','name','tip'])

display(df)

In [0]:
df.write.format("delta")\
        .mode("overwrite")\
        .option("overwriteSchema",True)\
        .save("/Volumes/deltalakeansh/default/deltavol/demosink/")